In [1]:
#just solution
import pandas as pd
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import cmudict
import os
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import string


In [6]:
# Read input.xlsx to get the list of URLs and their corresponding URL_IDs(upload you input file in home of jupyter)
input_file = "Input.xlsx"
df_input = pd.read_excel(input_file)

In [2]:
# Step 1: Data Extraction
def extract_article_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Check if 'h1' element exists before accessing its 'text' attribute
        article_title_element = soup.find('h1')
        if article_title_element:
            article_title = article_title_element.text.strip()
        else:
            article_title = "No Title Found"

        # Find the article text
        article_text_elements = soup.find_all('p')
        article_text = '\n'.join([p.text for p in article_text_elements])

        return article_title, article_text
    else:
        print(f"Error: Could not retrieve content from {url}")
        return None, None


#done

In [13]:
for index, row in df_input.iterrows():
    url_id = row["URL_ID"]
    url = row["URL"]
    article_title, article_text = extract_article_content(url)
    
    if article_title and article_text:
    # Save the article content in a text file
        with open(f"{url_id}.txt", "w", encoding="utf-8") as file:
            file.write(article_title + "\n")
            file.write(article_text)
#don't run just to check that we are getting the content in each text file or not(we are executing same in below)
#after few creating text just restart the kernel or you can see the 113 text files (3 files are 404 not found)

Error: Could not retrieve content from https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/


KeyboardInterrupt: 

In [15]:
# Step 2: Loading Stop Words 
#loaded the stopwords from folder
def load_stop_words_from_folder(folder_path, encodings=['utf-8', 'latin-1']):
    stop_words = set()
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        for encoding in encodings:
            try:
                with open(file_path, 'r', encoding=encoding) as file:
                    words = file.read().splitlines()
                    stop_words.update(words)
                break  # Stop if successful
            except UnicodeDecodeError:
                continue  # Try the next encoding if there's an error
    return stop_words
stop_words = load_stop_words_from_folder("StopWords")



In [17]:
stop_words   
#to check the stop words from (7)text file is store in this stop_words

{'KENDRICK',
 'MANA',
 'MAES',
 'SKILES',
 'KELLIE',
 'MERRIWEATHER',
 'JOANN',
 'BOHN',
 'LOVING',
 'GRANTHAM',
 'KRONE | Denmark ',
 'MCCOMAS',
 'KURTZ',
 'JACOBS',
 'WHIPPLE',
 'think',
 'JACOBSON',
 'DOOLITTLE',
 'RUNYAN',
 'says',
 'BRETON',
 'JAMIESON',
 'STRAUB',
 'gets',
 'other',
 'GIDGET',
 'MAURICE',
 'ARDOIN',
 'ALEMAN',
 'SWINK',
 'GITA',
 'WISCONSIN',
 'TAMEIKA',
 'FLORIN | Aruba ',
 'FRINK',
 'what',
 'BURKE',
 'HIMES',
 'PETE',
 'TREVINO',
 'SHANE',
 'GRAVES',
 'SERRATO',
 'SZABO',
 'RUDOLPH',
 'BARGER',
 'BELGIUM',
 'HARE',
 'LALIBERTE',
 'JAYME',
 'VANPELT',
 'ALISIA',
 'PRINCE',
 'PERRINE',
 'LUKER',
 'WHITWORTH',
 'BENNETT',
 'ROBERTO',
 'ELYSE',
 'LYLA',
 'FITZ',
 'FORTIER',
 'POE',
 'POND',
 'DENNA',
 'CUNDIFF',
 'MILAGRO',
 'KIMURA',
 'WOODBURY',
 'ROSENTHAL',
 'SHAD',
 'LYNETTE',
 'regarding',
 'GILBERTSON',
 'MARAGARET',
 'BUNN',
 'HARKINS',
 'NELDA',
 'JEFFREY',
 'FINE',
 'HERSHEL',
 'DALE',
 'RESENDEZ',
 'SOO',
 'ELI',
 'nearly',
 'SIEBERT',
 'NIKIA',
 'LEVIT

In [33]:
# Step 3: Text Cleaning 
clean = []
def clean_text(text, stop_words):
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Convert text to lowercase and split into words
    words = text.lower().split()
    
    # Exclude stop words
    cleaned_words = [word for word in words if word not in stop_words]
    
    # Join cleaned words back to a sentence
    cleaned_text = " ".join(cleaned_words)
    
    return cleaned_text  


In [9]:
# Step 4: Sentiment Analysis and Derived Variables

# Load the positive and negative words from the MasterDictionary folder
master_dictionary_folder = "MasterDictionary"
positive_words_file = "positive-words.txt"
negative_words_file = "negative-words.txt"
positive_words_path = os.path.join(master_dictionary_folder, positive_words_file)
negative_words_path = os.path.join(master_dictionary_folder, negative_words_file)
positive_words = load_stop_words(positive_words_path)
negative_words = load_stop_words(negative_words_path)

# Define a function to calculate the positive and negative scores

def calculate_positive_negative_scores(cleaned_text, positive_words, negative_words):
    positive_score = sum(1 for word in cleaned_text.split() if word in positive_words)
    negative_score = sum(1 for word in cleaned_text.split() if word in negative_words)
    return positive_score, negative_score

# Define a function to calculate the polarity score and subjectivity score
def calculate_polarity_subjectivity_scores(positive_score, negative_score, total_words_after_cleaning):
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (total_words_after_cleaning + 0.000001)
    return polarity_score, subjectivity_score

# Define a function to calculate the average sentence length
def calculate_average_sentence_length(cleaned_text):
    sentences = nltk.sent_tokenize(cleaned_text)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    total_sentences = len(sentences)
    return total_words / total_sentences

# Define a function to calculate the percentage of complex words
def calculate_complex_word_percentage(cleaned_text):
    words = word_tokenize(cleaned_text)
    complex_words = [word for word in words if count_syllables(word) > 2]
    return len(complex_words) / len(words)

# Define a function to calculate the Fog Index
def calculate_fog_index(average_sentence_length, complex_word_percentage):
    return 0.4 * (average_sentence_length + complex_word_percentage)

# Define a function to calculate the average number of words per sentence
def calculate_average_words_per_sentence(cleaned_text):
    sentences = nltk.sent_tokenize(cleaned_text)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    total_sentences = len(sentences)
    return total_words / total_sentences

# Define a function to count the number of syllables in a word
def count_syllables(word):
    d = cmudict.dict()
    if word.lower() in d:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    else:
        return 0

# Define a function to count the number of personal pronouns
def count_personal_pronouns(cleaned_text):
    pronoun_regex = r'\b(?:I|we|my|ours|us)\b'
    return len(re.findall(pronoun_regex, cleaned_text, re.IGNORECASE))

# Define a function to calculate the average word length
def calculate_average_word_length(cleaned_text):
    words = word_tokenize(cleaned_text)
    total_characters = sum(len(word) for word in words)
    return total_characters / len(words)


In [ ]:
# Calculate and save the derived variables for each article text
output_data = []

# Read input.xlsx to get the list of URLs and their corresponding URL_IDs
input_file = "Input.xlsx"
df_input = pd.read_excel(input_file)

# Loop through each row in df_input to extract article content and save in text files
for index, row in df_input.iterrows():
    url_id = row["URL_ID"]
    url = row["URL"]
    article_title, article_text = extract_article_content(url)

    if article_title and article_text:
        # Save the article content in a text file
        with open(f"{url_id}.txt", "w", encoding="utf-8") as file:
            file.write(article_title + "\n")
            file.write(article_text)

        # Read the content from the text file
        with open(f"{url_id}.txt", "r", encoding="utf-8") as file:
            text = file.read()

        # Clean the text using stop words
        cleaned_text = clean_text(text, stop_words)

        # Calculate the positive and negative scores
        positive_score, negative_score = calculate_positive_negative_scores(cleaned_text, positive_words, negative_words)

        # Calculate the total words after cleaning
        total_words_after_cleaning = len(cleaned_text.split())

        # Calculate the polarity score and subjectivity score
        polarity_score, subjectivity_score = calculate_polarity_subjectivity_scores(positive_score, negative_score, total_words_after_cleaning)

        # Calculate the average sentence length
        average_sentence_length = calculate_average_sentence_length(cleaned_text)

        # Calculate the percentage of complex words
        complex_word_percentage = calculate_complex_word_percentage(cleaned_text)

        # Calculate the Fog Index
        fog_index = calculate_fog_index(average_sentence_length, complex_word_percentage)

        # Calculate the average number of words per sentence
        average_words_per_sentence = calculate_average_words_per_sentence(cleaned_text)

        # Calculate the complex word count
        complex_word_count = len([word for word in cleaned_text.split() if count_syllables(word) > 2])

        # Calculate the word count
        word_count = len(cleaned_text.split())

        # Calculate the syllable count per word
        syllable_count_per_word = sum(count_syllables(word) for word in cleaned_text.split()) / word_count

        # Calculate the personal pronouns count
        personal_pronouns_count = count_personal_pronouns(cleaned_text)

        # Calculate the average word length
        average_word_length = calculate_average_word_length(cleaned_text)

        # Create a dictionary to store the derived variables for this URL
        url_data = {
            "URL_ID": url_id,
            "URL": url,
            "Positive Score": positive_score,
            "Negative Score": negative_score,
            "Polarity Score": polarity_score,
            "Subjectivity Score": subjectivity_score,
            "Avg Sentence Length": average_sentence_length,
            "Percentage of Complex Words": complex_word_percentage,
            "Fog Index": fog_index,
            "Avg Number of Words Per Sentence": average_words_per_sentence,
            "Complex Word Count": complex_word_count,
            "Word Count": word_count,
            "Syllable Per Word": syllable_count_per_word,
            "Personal Pronouns": personal_pronouns_count,
            "Avg Word Length": average_word_length
        }
        # Append the dictionary to the output_data list
        output_data.append(url_data)

# Step 5: Output Data to Excel

# Create a DataFrame from the output_data list
output_df = pd.DataFrame(output_data)

# Save the DataFrame to an Excel file
output_file = "Output.xlsx"
output_df.to_excel(output_file, index=False)

